In [ ]:
import numpy as np
import gymnasium
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import PolynomialFeatures  # makes poly features super easy

np.set_printoptions(precision=3, suppress=True)

In [2]:
# Notation for array sizes:
# - S: state dimensionality
# - D: features dimensionality
# - N: number of samples
#
# N is always the first dimension, meaning that states come in arrays of shape
# (N, S) and features in arrays of shape (N, D).
# We recommend to implement the functions below assuming that the input has
# always shape (N, S) and the output (N, D), even when N = 1.

def poly_features(state: np.array, degree: int) -> np.array:
    """
    Compute polynomial features. For example, if state = (s1, s2) and degree = 2,
    the output must be [1, s1, s2, s1*s2, s1**2, s2**2].
    """
    poly = PolynomialFeatures(degree = degree)
    feat = poly.fit_transform(state)
    return feat

def rbf_features(
    state: np.array,  # (N, S)
    centers: np.array,  # (D, S)
    sigmas: float,
) -> np.array:  # (N, D)
    """
    Computes exp(- ||state - centers||**2 / sigmas**2 / 2).
    """
    N, D = state.shape[0], centers.shape[0]
    state = np.repeat(state[:, np.newaxis, :], D, axis = 1) # (N, D, S)
    numer = -np.linalg.norm(state - centers, 2, -1) # (N, D)
    denom = 2 * (sigmas**2)
    feat = np.exp(numer / denom)
    assert feat.shape == (N, D)
    return feat


def tile_features(
    state: np.array,  # (N, S)
    centers: np.array,  # (D, S)
    widths: float,
    offsets: list = [0],  # list of tuples of length S
) -> np.array:  # (N, D)
    """
    Given centers and widths, you first have to get an array of 0/1, with 1s
    corresponding to tile the state belongs to.
    If "offsets" is passed, it means we are using multiple tilings, i.e., we
    shift the centers according to the offsets and repeat the computation of
    the 0/1 array. The final output will sum the "activations" of all tilings.
    We recommend to normalize the output in [0, 1] by dividing by the number of
    tilings (offsets).
    Recall that tiles are squares, so you can't use the L2 Euclidean distance to
    check if a state belongs to a tile, but the absolute distance.
    Note that tile coding is more general and allows for rectangles (not just squares)
    but let's consider only squares for the sake of simplicity. 
    """
    
    feats = []
    N, D = state.shape[0], centers.shape[0]
    state = np.repeat(state[:, np.newaxis, :], D, axis = 1) # (N, D, S)
        
    for offset in offsets:    
        
        dist = state - (centers + offset) # (N, D, S)
        dist = np.linalg.norm(dist, 1, -1)

        one_hot = np.argmin(dist, -1)
        closest_tile = np.zeros((N, D))
        closest_tile[np.arange(N), one_hot] = 1 # (N, D)
        
        inside_tile = 1 * (dist <= widths) # (N, D)
        
        feat = inside_tile * closest_tile
        
        assert (np.sum(feat, -1) <= 1).all() # Check for activation of multiple tiles
        
        feats.append(feat)
    
    feats = np.mean(feats, 0)
    
    return feats

def coarse_features(
    state: np.array,  # (N, S)
    centers: np.array,  # (D, S)
    widths: float,
    offsets: list = [0],  # list of tuples of length S
) -> np.array:  # (N, D)
    """
    Same as tile coding, but we use circles instead of squares, so use the L2
    Euclidean distance to check if a state belongs to a circle.
    Note that coarse coding is more general and allows for ellipses (not just circles)
    but let's consider only circles for the sake of simplicity.
    """

    feats = []
    N, D = state.shape[0], centers.shape[0]
    state = np.repeat(state[:, np.newaxis, :], D, axis = 1) # (N, D, S)
        
    for offset in offsets:    
        
        dist = state - (centers + offset) # (N, D, S)
        dist = np.sum(dist**2, -1) # (N, D)
        dist = np.sqrt(dist)
                
        feat = 1 * (dist <= widths) # (N, D)
                
        feats.append(feat)
    
    feats = np.mean(feats, 0)
    
    return feats

def aggregation_features(
    state: np.array,  # (N, S)
    centers: np.array,  # (D, S)
) -> np.array:  # (N, D)
    """
    Aggregate states to the closest center. The output will be an array of 0s and
    one 1 corresponding to the closest tile the state belongs to.
    Note that we can turn this into a discrete (finite) representation of the state,
    because we will have as many feature representations as centers.
    """
    
    N, D = state.shape[0], centers.shape[0]
    state = np.repeat(state[:, np.newaxis, :], D, axis = 1) # (N, D, S)
    
    dist = state - centers # (N, D, S)
    dist = np.sum(dist**2, -1) # (N, D)
    
    one_hot = np.argmin(dist, -1) # (N, )
    feats = np.zeros((N, D))
    feats[np.arange(N), one_hot] = 1 # (N, D)

    assert (np.sum(feats, -1) == 1).all() # Check for activation of multiple tiles

    return feats


In [3]:
state_size = 2
n_samples = 10

state = np.random.rand(n_samples, state_size)  # in [0, 1]

In [4]:
poly_deg = 2
poly = poly_features(state, 2)

In [ ]:
n_centers = 100

state_1_centers = np.linspace(-0.2, 1.2, n_centers)
state_2_centers = np.linspace(-0.2, 1.2, n_centers)
centers = np.array(
    np.meshgrid(state_1_centers, state_2_centers)
).reshape(state_size, -1).T  # makes a grid of uniformly spaced centers in the plane [-0.2, 1.2]^2
sigmas = 0.5
rbf = rbf_features(state, centers, sigmas)

extent = [
    state_1_centers[0],
    state_1_centers[-1],
    state_2_centers[0],
    state_2_centers[-1],
] 
plt.imshow(rbf[0].reshape(n_centers, n_centers), extent = extent, origin = 'lower')
plt.plot(state[0][0], state[0][1], marker = "+", markersize = 12, color = "red")

plt.title("RBF")
plt.show()

In [ ]:
n_centers = 10

state_1_centers = np.linspace(-0.2, 1.2, n_centers)
state_2_centers = np.linspace(-0.2, 1.2, n_centers)
centers = np.array(
    np.meshgrid(state_1_centers, state_2_centers)
).reshape(state_size, -1).T  # makes a grid of uniformly spaced centers in the plane [-0.2, 1.2]^2
sigmas = 0.3
widths = 0.14 # NON-OVERLAPPING TILES - (max val = 1.4 / n_centers)
offsets = [(-0.1, 0.0), (0.0, 0.1), (0.1, 0.0), (0.0, -0.1)]
tile_one = tile_features(state, centers, widths)

extent = [
    state_1_centers[0],
    state_1_centers[-1],
    state_2_centers[0],
    state_2_centers[-1],
] 
plt.imshow(tile_one[0].reshape(n_centers, n_centers), extent = extent, origin = 'lower')
plt.plot(state[0][0], state[0][1], marker = "+", markersize = 12, color = "red")


plt.title("Tile Coding (1 Tile)")
plt.show()

In [ ]:
n_centers = 10

state_1_centers = np.linspace(-0.2, 1.2, n_centers)
state_2_centers = np.linspace(-0.2, 1.2, n_centers)
centers = np.array(
    np.meshgrid(state_1_centers, state_2_centers)
).reshape(state_size, -1).T  # makes a grid of uniformly spaced centers in the plane [-0.2, 1.2]^2
sigmas = 0.3
widths = 0.14 # NON-OVERLAPPING TILES - (max val = 14 / n_centers)
offsets = [(-widths/2, 0.0), (0.0, widths/2), (widths/2, 0.0), (0.0, -widths/2)]
tile_multi = tile_features(state, centers, widths, offsets)

extent = [
    state_1_centers[0],
    state_1_centers[-1],
    state_2_centers[0],
    state_2_centers[-1],
] 
plt.imshow(tile_multi[0].reshape(n_centers, n_centers), extent = extent, origin = 'lower')
plt.plot(state[0][0], state[0][1], marker = "+", markersize = 12, color = "red")


plt.title("Tile Coding (4 Tiles)")
plt.show()

In [ ]:
n_centers = 100

state_1_centers = np.linspace(-0.2, 1.2, n_centers)
state_2_centers = np.linspace(-0.2, 1.2, n_centers)
centers = np.array(
    np.meshgrid(state_1_centers, state_2_centers)
).reshape(state_size, -1).T  # makes a grid of uniformly spaced centers in the plane [-0.2, 1.2]^2
sigmas = 0.3
widths = 0.2
offsets = [(-0.1, 0.0), (0.0, 0.1), (0.1, 0.0), (0.0, -0.1)]
coarse_one = coarse_features(state, centers, widths)

extent = [
    state_1_centers[0],
    state_1_centers[-1],
    state_2_centers[0],
    state_2_centers[-1],
] 
plt.imshow(coarse_one[0].reshape(n_centers, n_centers), extent = extent, origin = 'lower')
plt.plot(state[0][0], state[0][1], marker = "+", markersize = 12, color = "red")


plt.title("Coarse Coding (1 Field)")
plt.show()

In [ ]:
n_centers = 100

state_1_centers = np.linspace(-0.2, 1.2, n_centers)
state_2_centers = np.linspace(-0.2, 1.2, n_centers)
centers = np.array(
    np.meshgrid(state_1_centers, state_2_centers)
).reshape(state_size, -1).T  # makes a grid of uniformly spaced centers in the plane [-0.2, 1.2]^2
sigmas = 0.3
widths = 0.2
offsets = [(-0.1, 0.0), (0.0, 0.1), (0.1, 0.0), (0.0, -0.1)]
coarse_multi = coarse_features(state, centers, widths, offsets)

extent = [
    state_1_centers[0],
    state_1_centers[-1],
    state_2_centers[0],
    state_2_centers[-1],
] 
plt.imshow(coarse_multi[0].reshape(n_centers, n_centers), extent = extent, origin = 'lower')
plt.plot(state[0][0], state[0][1], marker = "+", markersize = 12, color = "red")

plt.title("Coarse Coding (4 Fields)")
plt.show()

In [ ]:
n_centers = 100

state_1_centers = np.linspace(-0.2, 1.2, n_centers)
state_2_centers = np.linspace(-0.2, 1.2, n_centers)
centers = np.array(
    np.meshgrid(state_1_centers, state_2_centers)
).reshape(state_size, -1).T  # makes a grid of uniformly spaced centers in the plane [-0.2, 1.2]^2
aggr = aggregation_features(state, centers)

extent = [
    state_1_centers[0],
    state_1_centers[-1],
    state_2_centers[0],
    state_2_centers[-1],
] 
plt.imshow(aggr[0].reshape(n_centers, n_centers), extent = extent, origin = 'lower')
plt.plot(state[0][0], state[0][1], marker = "+", markersize = 12, color = "red")

plt.title('Aggreg.')
plt.show()

In [ ]:
state[0]

In [ ]:
#################### PART 2
# Consider the function below.

x = np.linspace(-10, 10, 100)
y = np.sin(x) + x**2 - 0.5 * x**3 + np.log(np.abs(x))
fig, axs = plt.subplots(1, 1)
axs.plot(x, y)
plt.show()

In [27]:
n_centers = 100
state = np.random.rand(n_samples, state_size)  # in [0, 1]

state_1_centers = np.linspace(-10.5, 10.5, n_centers)
state_2_centers = np.linspace(-10.5, 10.5, n_centers)
centers = np.array(
    np.meshgrid(state_1_centers, state_2_centers)
).reshape(state_size, -1).T  # makes a grid of uniformly spaced centers in the plane [-0.2, 1.2]^2
sigmas = 0.3
widths = 0.21
offsets = [(-0.1, 0.0), (0.0, 0.1), (0.1, 0.0), (0.0, -0.1)]

In [28]:
n_centers = 200

state_1_centers = np.linspace(-10.5, 10.5, n_centers)
state_2_centers = np.linspace(-10.5, 10.5, n_centers)
agg_centers = np.array(
    np.meshgrid(state_1_centers, state_2_centers)
).reshape(state_size, -1).T  # makes a grid of uniformly spaced centers in the plane [-0.2, 1.2]^2


In [ ]:
max_iter = 10000
thresh = 1e-8

for name, get_phi in zip(["Poly", "RBFs", "Tiles", "Coarse", "Aggreg."], [
        lambda state : poly_features(state, 3),
        lambda state : rbf_features(state, centers, sigmas),
        lambda state : tile_features(state, centers, widths, offsets),
        lambda state : coarse_features(state, centers, widths, offsets),
        lambda state : aggregation_features(state, agg_centers),
    ]):
    
    if name == 'Poly':
        alpha = 1e-5
    else:
        alpha = 5.0
    phi = get_phi(x[..., None])  # from (N,) to (N, S) with S = 1
    weights = np.zeros(phi.shape[-1])
    pbar = tqdm(total=max_iter)
    for iter in range(max_iter):
        # do gradient descent
        y_hat = np.dot(phi, weights)
        diff = y_hat - y
        mse = np.mean(diff**2)
        diff = np.repeat(diff[:, np.newaxis], phi.shape[-1], axis = -1)
        weights = weights - alpha * np.mean(diff * phi, 0)
        pbar.set_description(f"MSE: {mse}")
        pbar.update()
        if mse < thresh:
            break
        
    
    print(f"Iterations: {iter}, MSE: {mse}, N. of Features {len(weights)}")
    fig, axs = plt.subplots(1, 2)
    axs[0].plot(x, y)
    axs[1].plot(x, y_hat)
    axs[0].set_title("True Function")
    axs[1].set_title(f"Approximation with {name} (MSE {mse:.3f})")
    plt.suptitle(f"FA: {name}")
    plt.show()

In [ ]:
x = np.linspace(-10, 10, 100)
y = np.zeros(x.shape)
y[0:10] = x[0:10]**3 / 3.0
y[10:20] = np.exp(x[25:35])
y[20:30] = -x[0:10]**3 / 2.0
y[30:60] = 100.0
y[60:70] = 0.0
y[70:100] = np.cos(x[70:100]) * 100.0
fig, axs = plt.subplots(1, 1)
axs.plot(x, y)
plt.show()

In [32]:
n_centers = 100
state = np.random.rand(n_samples, state_size)  # in [0, 1]

state_1_centers = np.linspace(-10.5, 10.5, n_centers)
state_2_centers = np.linspace(-10.5, 10.5, n_centers)
centers = np.array(
    np.meshgrid(state_1_centers, state_2_centers)
).reshape(state_size, -1).T  # makes a grid of uniformly spaced centers in the plane [-0.2, 1.2]^2
sigmas = 0.3
widths = 0.21
offsets = [(-0.1, 0.0), (0.0, 0.1), (0.1, 0.0), (0.0, -0.1)]

In [33]:
n_centers = 200

state_1_centers = np.linspace(-10.5, 10.5, n_centers)
state_2_centers = np.linspace(-10.5, 10.5, n_centers)
agg_centers = np.array(
    np.meshgrid(state_1_centers, state_2_centers)
).reshape(state_size, -1).T  # makes a grid of uniformly spaced centers in the plane [-0.2, 1.2]^2


In [ ]:
# Now repeat the experiment but fit the following function y.
# Submit your plots and discuss your results, paying attention to the
# non-smoothness of the new target function.
# - How did you change your hyperparameters? Did you use more/less wider/narrower features?
# - Consider the number of features. How would it change if your state would be 2-dimensional?
# Discuss each bullet point in at most two sentences.


max_iter = 10000
thresh = 1e-8

for name, get_phi in zip(["Poly", "RBFs", "Tiles", "Coarse", "Aggreg."], [
        lambda state : poly_features(state, 3),
        lambda state : rbf_features(state, centers, sigmas),
        lambda state : tile_features(state, centers, widths, offsets),
        lambda state : coarse_features(state, centers, widths, offsets),
        lambda state : aggregation_features(state, agg_centers),
    ]):
    
    if name == 'Poly':
        alpha = 1e-5
    else:
        alpha = 5.0
    phi = get_phi(x[..., None])  # from (N,) to (N, S) with S = 1
    weights = np.zeros(phi.shape[-1])
    pbar = tqdm(total=max_iter)
    for iter in range(max_iter):
        # do gradient descent
        y_hat = np.dot(phi, weights)
        diff = y_hat - y
        mse = np.mean(diff**2)
        diff = np.repeat(diff[:, np.newaxis], phi.shape[-1], axis = -1)
        weights = weights - alpha * np.mean(diff * phi, 0)
        pbar.set_description(f"MSE: {mse}")
        pbar.update()
        if mse < thresh:
            break
        
    
    print(f"Iterations: {iter}, MSE: {mse}, N. of Features {len(weights)}")
    fig, axs = plt.subplots(1, 2)
    axs[0].plot(x, y)
    axs[1].plot(x, y_hat)
    axs[0].set_title("True Function")
    axs[1].set_title(f"Approximation with {name} (MSE {mse:.3f})")
    plt.suptitle(f"FA: {name}")
    plt.show()


In [ ]:
#################### PART 3
# Consider the Gridworld depicted below. The dataset below contains episodes
# collected using the optimal policy, and the heatmap below shows its V-function.
# - Consider the 5 FAs implemented above and discuss why each would be a
#   good/bad choice. Discuss each in at most two sentences.

# The given data is a dictionary of (s, a, r, s', term, Q) arrays.
# Unlike the previous assignments, the state is the (x, y) coordinate of the agent
# on the grid.
# - Run batch semi-gradient TD prediction with a FA of your choice (the one you
#   think would work best) to learn an approximation of the V-function.
#   Use gamma = 0.99. Increase the number of iterations, if you'd like.
#   Plot your result of the true V-function against your approximation using the
#   provided plotting function.

data = np.load("a6_gridworld.npz")
s = data["s"]
a = data["a"]
r = data["r"]
s_next = data["s_next"]
Q = data["Q"]
V = data["Q"].max(-1)  # value of the greedy policy
term = data["term"]
n = s.shape[0]
gamma = 0.99

fig, axs = plt.subplots(1, 1)
axs.tricontourf(s[:, 0], s[:, 1], V, levels = 100)
plt.show()

In [37]:
n_actions = len(np.unique(a))

In [38]:

# RBF
n_centers = 10
sigmas = 0.5


# Tile-Coding
# n_centers = 10
# widths = 8/n_centers


state_1_centers = np.linspace(0, 8, n_centers)
state_2_centers = np.linspace(0, 8, n_centers)
centers = np.array(
    np.meshgrid(state_1_centers, state_2_centers)
).reshape(state_size, -1).T  # makes a grid of uniformly spaced centers in the plane [-0.2, 1.2]^2

offsets = []
for i in range(0, 4):
    for j in range(0, 4):
    
        if i != 0 and j != 0:
            offsets.append((i/2, j/2))


# Pick one
# name, get_phi = "Poly", lambda state : poly_features(state, 3)
name, get_phi = "RBFs", lambda state : rbf_features(state, centers, sigmas)
# name, get_phi = "Tiles", lambda state : tile_features(state, centers, widths, offsets)
# name, get_phi = "Tiles", lambda state : coarse_features(state, centers, widths, offsets)
# name, get_phi = "Aggreg.", lambda state: aggregation_features(state, ...)


phi = get_phi(s)
phi_next = get_phi(s_next)

In [ ]:
extent = [
    state_1_centers[0],
    state_1_centers[-1],
    state_2_centers[0],
    state_2_centers[-1],
] 
plt.imshow(phi[0].reshape(n_centers, n_centers), extent = extent, origin = 'lower')
plt.plot(s[0][0], s[0][1], marker = "+", markersize = 12, color = "red")

In [ ]:
max_iter = 25000
thresh = 1e-5

alpha = 1e-1
alpha_decay = 0.999

weights = np.zeros(phi.shape[-1])
# pbar = tqdm(total=max_iter)
for iter in range(max_iter):
    
    # alpha = max(alpha * alpha_decay, 1e-4)
        
    v_hat = np.dot(phi, weights)
    v_next_hat = np.dot(phi_next, weights)
    td_error = (r + gamma * v_next_hat * (1-term)) - v_hat
    td_error_braodcased = np.repeat(td_error[:, np.newaxis], phi.shape[-1], axis = -1)
    weights = weights + alpha * np.mean(td_error_braodcased * phi, 0)

    mse = np.mean((V - v_hat)**2)
    # pbar.set_description(f"TDE: {np.mean(np.abs(td_error))}, MSE: {mse}")
    # pbar.update()
    if iter % 100 == 0:
        print(iter, mse)
    if mse < thresh:
        break

# print(f"Iterations: {iter}, MSE: {mse}, N. of Features {len(weights)}")
# fig, axs = plt.subplots(1, 2)
# axs[0].tricontourf(s[:, 0], s[:, 1], V, levels = 100)
# axs[1].tricontourf(s[:, 0], s[:, 1], v_hat, levels = 100)
# axs[0].set_title("True Function")
# axs[1].set_title(f"Approximation with ??? (MSE {mse:.3f})")
# plt.show()


In [ ]:
fig, axs = plt.subplots(1, 2)
axs[0].tricontourf(s[:, 0], s[:, 1], V, levels = 100)
axs[1].tricontourf(s[:, 0], s[:, 1], v_hat, levels = 100)
axs[0].set_title("True Function")
axs[1].set_title(f"Approximation with RBF (MSE {mse:.3f})")
plt.show()

In [ ]:
#################### PART 3
# Consider the Gridworld depicted below. The dataset below contains episodes
# collected using the optimal policy, and the heatmap below shows its V-function.
# - Consider the 5 FAs implemented above and discuss why each would be a
#   good/bad choice. Discuss each in at most two sentences.

# The given data is a dictionary of (s, a, r, s', term, Q) arrays.
# Unlike the previous assignments, the state is the (x, y) coordinate of the agent
# on the grid.
# - Run batch semi-gradient TD prediction with a FA of your choice (the one you
#   think would work best) to learn an approximation of the V-function.
#   Use gamma = 0.99. Increase the number of iterations, if you'd like.
#   Plot your result of the true V-function against your approximation using the
#   provided plotting function.

data = np.load("a6_gridworld.npz")
s = data["s"]
a = data["a"]
r = data["r"]
s_next = data["s_next"]
Q = data["Q"]
V = data["Q"].max(-1)  # value of the greedy policy
term = data["term"]
n = s.shape[0]
gamma = 0.99

fig, axs = plt.subplots(1, 1)
axs.tricontourf(s[:, 0], s[:, 1], V, levels = 100)
plt.show()

In [75]:
# RBF
n_centers = 10
sigmas = 1


# # Tile-Coding
# n_centers = 10
# widths = 8/n_centers


state_1_centers = np.linspace(0, 8, n_centers)
state_2_centers = np.linspace(0, 8, n_centers)
centers = np.array(
    np.meshgrid(state_1_centers, state_2_centers)
).reshape(state_size, -1).T  # makes a grid of uniformly spaced centers in the plane [-0.2, 1.2]^2

offsets = []
for i in range(0, 4):
    for j in range(0, 4):
    
        if i != 0 and j != 0:
            offsets.append((i/2, j/2))


# Pick one
# name, get_phi = "Poly", lambda state : poly_features(state, 3)
name, get_phi = "RBFs", lambda state : rbf_features(state, centers, sigmas)
# name, get_phi = "Tiles", lambda state : tile_features(state, centers, widths, offsets)
# name, get_phi = "Tiles", lambda state : coarse_features(state, centers, widths, offsets)
# name, get_phi = "Aggreg.", lambda state: aggregation_features(state, ...)


In [ ]:
#################### PART 4
# - Run TD again, but this time learn an approximation of the Q-function.
#   How did you have to change your code?

max_iter = 25000
alpha = 1e-5
thresh = 0.1035

phi = get_phi(s)
phi_broadcasted = np.repeat(phi[:, np.newaxis, :], n_actions, axis = 1)
phi_next = get_phi(s_next)
phi_next_broadcasted = np.repeat(phi_next[:, np.newaxis, :], n_actions, axis = 1)

# weights = np.random.randn(n_actions, phi.shape[-1])  # you have to change something here
weights = np.zeros((n_actions, phi.shape[-1]))  # you have to change something here
pbar = tqdm(total=max_iter)

for iter in range(max_iter):
    
    tot_td_error = 0

    q_hat = np.sum(phi * weights[a], 1)
    q_next_hat = np.max(np.dot(phi_next, weights.T), 1)
    td_error = r + (gamma * q_next_hat * (1 - term)) - q_hat
    
    for action in range(n_actions):
        mask = (a == action)
        weights[action] += alpha * np.dot(phi[mask].T, td_error[mask])
        
    q_estimate = np.dot(phi_next, weights.T)
    mse = np.mean((q_estimate - Q)**2)

    if iter % 10 == 0:
        print(iter, mse)
        
    if mse < thresh:
        break

In [ ]:
Q_hat = np.sum(phi_broadcasted * weights, -1)

print(f"Iterations: {iter}, MSE: {mse}, N. of Features {len(weights)}")
fig, axs = plt.subplots(5, 2, figsize = (7, 15))
for i in range(5):
    axs[i][0].tricontourf(s[:, 0], s[:, 1], Q[:, i], levels=100)
    axs[i][1].tricontourf(s[:, 0], s[:, 1], Q_hat[:, i], levels=100)  # depends on how you implemented it
    axs[i][0].set_title("True Function")
    axs[i][1].set_title(f"Approximation with RBF (MSE {mse:.3f})")
plt.show()